In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import json
import glob

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

In [2]:
datestamp = '20220901'

In [3]:
pda_source_dir = '/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior Views/'
nopda_source_dir = '/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superior Views/'
export_dir = f'/zfs/wficai/pda/model_data/{datestamp}/'

if not os.path.exists(export_dir):
    os.makedirs(export_dir, exist_ok=True)

pda_label_jsons = ['/zfs/wficai/pda/model_data/20220711_PDA_small_incomplete_batch.json', '/zfs/wficai/pda/model_data/pda_export-2022-09-01T00 27 33.192Z.json']
nopda_label_jsons = ['/zfs/wficai/pda/model_data/nonpda_export-2022-09-01T00 26 59.116Z.json']

In [4]:
res_x = 224
res_y = 224

# Process source video data

In [5]:
pda_vids = [{'patient_type': 'pda', 'mp4_path': path} for path in glob.glob(f"{pda_source_dir}/*.mp4")]
nopda_vids = [{'patient_type': 'nopda', 'mp4_path': path} for path in glob.glob(f"{nopda_source_dir}/*.mp4")]
df_vids = pd.DataFrame(pda_vids+nopda_vids)
df_vids['external_id'] = ["_".join(fname[-1].split('_')[:2]) for fname in df_vids.mp4_path.str.split('/')]
df_vids

,patient_type,mp4_path,external_id
0,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...,study26_dicom51
1,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...,study54_dicom98
2,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...,study48_dicom136
3,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...,study22_dicom70
4,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...,study19_dicom17
...,...,...,...
5154,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...,study61_dicom49
5155,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...,study34_dicom78
5156,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...,study24_dicom113
5157,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...,study55_dicom122


In [6]:
df_vids.groupby('patient_type').size()

patient_type
nopda    2468
pda      2691
dtype: int64

# Process label data

In [7]:
# load the json data
labels = []
for path in pda_label_jsons + nopda_label_jsons:
    with open(path, 'r') as f:
        labels += json.load(f)

In [8]:
# parse
type_map = {
    "PDA Classification -- Non-PDA Views": "nopda",
    "PDA Classification": "pda",
    "PDA Classification -- PDA Views": "pda",
}

labels_parsed = []
for lab in labels:
    if type(lab['Label'])==dict:
        res = lab['Label']
        res['external_id'] = lab['External ID']
        res['study'] = lab['External ID'].split('_')[0]
        res['patient_type'] = type_map[lab['Project Name']]
        labels_parsed.append(res)

In [9]:
df_labs = pd.DataFrame(labels_parsed)
df_labs

,view,mode,diagnosis,external_id,study,patient_type
0,pdaView,2d,[],study1_dicom8,study1,pda
1,pdaView,2d,[],study1_dicom10,study1,pda
2,pdaView,color,[],study1_dicom11,study1,pda
3,nonPDAView,2d,NaN,study1_dicom1,study1,pda
4,nonPDAView,2d,[],study1_dicom2,study1,pda
...,...,...,...,...,...,...
2522,nonPDAView,2d,[],study45_dicom54,study45,nopda
2523,pdaRelatedView,color,[],study57_dicom101,study57,nopda
2524,nonPDAView,2d,[],study9_dicom11,study9,nopda
2525,pdaRelatedView,color_compare,[],study24_dicom110,study24,nopda


In [10]:
# ensure uniqueness of external id among each patient_type
assert not (df_labs.groupby(['patient_type', 'external_id']).size()>1).any(), "patient_type-external_id pairs must be unique"

In [11]:
df_labs.groupby(['patient_type']).size()

patient_type
nopda    1238
pda      1289
dtype: int64

In [12]:
df_labs.groupby(['view']).size()

view
nonPDAView        1654
pdaRelatedView     508
pdaView            361
dtype: int64

In [13]:
df_labs.groupby(['mode']).size()

mode
2d               1092
color             850
color_compare     583
dtype: int64

In [14]:
df_labs.groupby(['patient_type', 'view', 'mode']).size()

patient_type  view            mode         
nopda         nonPDAView      2d               406
                              color            275
                              color_compare    190
              pdaRelatedView  2d                98
                              color             85
                              color_compare     59
              pdaView         2d                32
                              color             41
                              color_compare     51
pda           nonPDAView      2d               383
                              color            285
                              color_compare    115
              pdaRelatedView  2d               104
                              color             96
                              color_compare     66
              pdaView         2d                68
                              color             68
                              color_compare    101
dtype: int64

In [15]:
df_labs.query('view!="nonPDAView" and mode!="2d"')

,view,mode,diagnosis,external_id,study,patient_type
2,pdaView,color,[],study1_dicom11,study1,pda
8,pdaView,color,[],study1_dicom12,study1,pda
9,pdaView,color,[],study1_dicom15,study1,pda
11,pdaRelatedView,color,[],study1_dicom17,study1,pda
20,pdaRelatedView,color,[],study1_dicom29,study1,pda
...,...,...,...,...,...,...
2499,pdaView,color,[],study37_dicom84,study37,nopda
2519,pdaView,color,[],study44_dicom56,study44,nopda
2521,pdaRelatedView,color_compare,[],study40_dicom77,study40,nopda
2523,pdaRelatedView,color,[],study57_dicom101,study57,nopda


In [16]:
df_labs.query('view!="nonPDAView" and mode!="2d"').groupby(['patient_type']).size()

patient_type
nopda    236
pda      334
dtype: int64

# Merge datasets

In [17]:
df_model = df_labs.merge(df_vids, how='inner')

In [18]:
df_model

,view,mode,diagnosis,external_id,study,patient_type,mp4_path
0,pdaView,2d,[],study1_dicom8,study1,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
1,pdaView,2d,[],study1_dicom10,study1,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
2,pdaView,color,[],study1_dicom11,study1,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
3,nonPDAView,2d,NaN,study1_dicom1,study1,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
4,nonPDAView,2d,[],study1_dicom2,study1,pda,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
...,...,...,...,...,...,...,...
2504,nonPDAView,2d,[],study45_dicom54,study45,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...
2505,pdaRelatedView,color,[],study57_dicom101,study57,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...
2506,nonPDAView,2d,[],study9_dicom11,study9,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...
2507,pdaRelatedView,color_compare,[],study24_dicom110,study24,nopda,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...


# Preprocess frames

In [19]:
def my_line(x_coord, slope, intercept):
    return x_coord*slope + intercept

def mask_image(img, slope, intercept):
    img = img.clone()

    # list all x and y coordinate indices
    y_coordinates = torch.arange(0, img.shape[-2])
    x_coordinates = torch.arange(0, img.shape[-1])

    # compute the y-values associated with given slope and intercept
    # for each x coordinate
    y_line_points = my_line(x_coordinates, slope=slope, intercept=intercept)

    # locate all pixels with y coordinates less than line values
    mask = y_coordinates[:,None] < y_line_points 

    # make symmetric mask by flipping
    mask = mask | mask.flip(-1)

    # expand mask dimensions to match image dimensions
    expanded_mask = mask[None].expand(img.shape[0],img.shape[1],-1, -1)

    # zero out image in masked regions
    img[expanded_mask] = 0

    return img

In [20]:
def preproc(frames):

    frames = frames.permute(0,3,1,2)
    
    #Resize
    splitFrames = torch.split(frames, 10)
    resizedFrames = []
    aspectRatio = (frames.shape[3])/(frames.shape[2])
    for f in splitFrames:
        h = int(1.2 * res_y)
        w = int(h*aspectRatio)
        f = TF.resize(f, (h, w))
        f = TF.center_crop(f, (res_x, res_y))
        
        resizedFrames.append(f)

    resizedFrames = torch.concat(resizedFrames)
    
    #Image Masking
    yIntercept = int(resizedFrames.shape[2]/3.2)
    finalResult = mask_image(resizedFrames, -2, yIntercept)
    
    #tor
    return finalResult

In [ ]:
video_metadata = []
for ix, row in df_model.iterrows():
    print(" "*100, end='\r')
    print(f"Converting video {ix+1} of {len(df_model)}.", end="\r")
    
    frames, _, _ =  torchvision.io.read_video(row['mp4_path'])
    
    # store some video metadata
    video_metadata.append({'external_id': row['external_id'], 'patient_type': row['patient_type'], 'num_frames': frames.shape[0]})
    
    frames = preproc(frames)
    
    # save individual frames
    for ix, f in enumerate(frames):
        filename = f"{export_dir}{row['patient_type']}_{row['external_id']}_frame{str(ix).zfill(4)}.jpg"
        torchvision.io.write_jpeg(f, filename)

/home/spandir/.conda/envs/pda/lib/python3.9/site-packages/torchvision/io/video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


RuntimeError: Error opening output file

In [20]:
df_vid_meta = pd.DataFrame(video_metadata)
df_vid_meta

,external_id,patient_type,num_frames
0,study1_dicom8,pda,134
1,study1_dicom10,pda,100
2,study1_dicom11,pda,70
3,study1_dicom1,pda,109
4,study1_dicom2,pda,72
...,...,...,...
2504,study45_dicom54,nopda,105
2505,study57_dicom101,nopda,49
2506,study9_dicom11,nopda,107
2507,study24_dicom110,nopda,147


In [21]:
df_vid_meta = df_vid_meta.rename(columns={'patient_id': 'patient_type'})
df_vid_meta

,external_id,patient_type,num_frames
0,study1_dicom8,pda,134
1,study1_dicom10,pda,100
2,study1_dicom11,pda,70
3,study1_dicom1,pda,109
4,study1_dicom2,pda,72
...,...,...,...
2504,study45_dicom54,nopda,105
2505,study57_dicom101,nopda,49
2506,study9_dicom11,nopda,107
2507,study24_dicom110,nopda,147


# Prepare CSV data tables

We will prepare the following metadata tables:
1. study.csv -- each row corresponds to a particular study. We need this in order to correctly split studies into train/test partitions. Indexed by `patient_type`-`study` pairs. Note: `study` alone is not unique.
2. video.csv -- each row corresponds to a particular video from a study. We use this to store the video metadata since our annotations are at the video level. `patient_type`-`external_id` pairs. 
3. frame.csv -- each row corresponds to a particular frame from a video. This contains the filepaths to the frame images. Can be merged with video on `patient_type`-`external_id`.

### 3. frame.csv

In [22]:
frames = glob.glob(export_dir + '*.jpg')

def parse_path(fp):
    fn = fp.split('/')[-1]
    
    return dict(
        patient_type = fn.split('_')[0],
        external_id = '_'.join(fn.split('_')[1:3]),
        png_path = fp
    )

df_frames = pd.DataFrame([parse_path(p) for p in frames])

df_frames

,patient_type,external_id,png_path
0,pda,study22_dicom80,/zfs/wficai/pda/model_data/20220901/pda_study2...
1,nopda,study37_dicom77,/zfs/wficai/pda/model_data/20220901/nopda_stud...
2,pda,study10_dicom67,/zfs/wficai/pda/model_data/20220901/pda_study1...
3,pda,study3_dicom15,/zfs/wficai/pda/model_data/20220901/pda_study3...
4,pda,study7_dicom2,/zfs/wficai/pda/model_data/20220901/pda_study7...
...,...,...,...
227200,pda,study42_dicom18,/zfs/wficai/pda/model_data/20220901/pda_study4...
227201,nopda,study42_dicom79,/zfs/wficai/pda/model_data/20220901/nopda_stud...
227202,nopda,study9_dicom26,/zfs/wficai/pda/model_data/20220901/nopda_stud...
227203,nopda,study6_dicom69,/zfs/wficai/pda/model_data/20220901/nopda_stud...


In [23]:
df_frames.to_csv(f'/zfs/wficai/pda/model_data/{datestamp}_frame.csv', index=False)

### 2. video.csv

In [24]:
df_video = df_vid_meta.merge(df_model, how='inner')

# drop if missing view or mode
df_video = df_video.dropna(subset=['view', 'mode'])

df_video

,external_id,patient_type,num_frames,view,mode,diagnosis,study,mp4_path
0,study1_dicom8,pda,134,pdaView,2d,[],study1,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
1,study1_dicom10,pda,100,pdaView,2d,[],study1,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
2,study1_dicom11,pda,70,pdaView,color,[],study1,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
3,study1_dicom1,pda,109,nonPDAView,2d,NaN,study1,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
4,study1_dicom2,pda,72,nonPDAView,2d,[],study1,/zfs/wficai/pda/batch_1/PDA_Batch_1/Superior V...
...,...,...,...,...,...,...,...,...
2504,study45_dicom54,nopda,105,nonPDAView,2d,[],study45,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...
2505,study57_dicom101,nopda,49,pdaRelatedView,color,[],study57,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...
2506,study9_dicom11,nopda,107,nonPDAView,2d,[],study9,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...
2507,study24_dicom110,nopda,147,pdaRelatedView,color_compare,[],study24,/zfs/wficai/pda/batch_1/Non-PDA_Batch_1/Superi...


In [25]:
df_video['mode'].value_counts()

2d               1087
color             849
color_compare     569
Name: mode, dtype: int64

In [26]:
df_video.to_csv(f'/zfs/wficai/pda/model_data/{datestamp}_video.csv', index=False)

### 1. study.csv

In [27]:
# construct a study table. 
df_study = df_video.groupby(['patient_type', 'study'], as_index=False).agg({'external_id': 'nunique', 'num_frames': 'sum'}).rename(columns={'external_id': 'num_videos'})
df_study

,patient_type,study,num_videos,num_frames
0,nopda,study1,12,610
1,nopda,study10,19,1370
2,nopda,study11,8,595
3,nopda,study12,7,413
4,nopda,study13,13,1006
...,...,...,...,...
116,pda,study5,48,5539
117,pda,study6,30,2102
118,pda,study7,24,2392
119,pda,study8,46,4244


In [28]:
df_study.to_csv(f'/zfs/wficai/pda/model_data/{datestamp}_study.csv', index=False)